In [2]:
import numpy as np
import pandas as pd

In [34]:
df = pd.read_csv('./data/Credit Card Fraud Detection/creditcard.csv')

missing_values = df.isnull().sum()

# Check if there are any missing values
if missing_values.any():
    print("There are missing values in the dataset.")
    # Display the count of missing values for each column
    print("Missing values per column:")
    print(missing_values)
else:
    print("There are no missing values in the dataset.")



There are no missing values in the dataset.


In [55]:
df = pd.read_csv('./data/Telco Customer Churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')
# df.head()

In [ ]:
df.describe(include='all')

In [59]:

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
# errors='coerce' is a good approach to handle non-numeric values by replacing them with NaN.


missing_values = df.isna().sum()
# print(missing_values)

# Check if there are any missing values
if missing_values.any():
    print("There are missing values in the dataset.")
    # Display the count of missing values for each column
    # find the columns with missing values
    columns_with_missing_values = df.columns[missing_values > 0]
    print("Missing values per column:")
    print(missing_values[columns_with_missing_values])

    # fix the missing values
    for column in columns_with_missing_values:
        if df[column].dtype == np.float64:
            df[column].fillna(df[column].mean(), inplace=True)
        else:
            df[column].fillna(df[column].mode()[0], inplace=True)
            df[column].fillna(df[column].mode()[0], inplace=True)
    
    print("Missing values per column are fixed")


else:
    print("There are no missing values in the dataset.")

# df.info()

There are no missing values in the dataset.


In [53]:
# check if there are any null values
print(df.isnull().values.any())
df.describe(include='all').transpose()

False


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
customerID,7043,7043,7590-VHVEG,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gender,7043,2,Male,3555,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SeniorCitizen,7043.0,NaN,NaN,NaN,0.162147,0.368612,0.0,0.0,0.0,0.0,1.0
Partner,7043,2,No,3641,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dependents,7043,2,No,4933,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tenure,7043.0,NaN,NaN,NaN,32.371149,24.559481,0.0,9.0,29.0,55.0,72.0
PhoneService,7043,2,Yes,6361,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MultipleLines,7043,3,No,3390,NaN,NaN,NaN,NaN,NaN,NaN,NaN
InternetService,7043,3,Fiber optic,3096,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OnlineSecurity,7043,3,No,3498,NaN,NaN,NaN,NaN,NaN,NaN,NaN
